In [ ]:
import torch
import torch.nn as nn

In [ ]:
class GaussianMixtureModel(nn.Module):
    def __init__(self, n_components, n_features):
        super(GaussianMixtureModel, self).__init__()
        self.n_components = n_components
        self.n_features = n_features
        
        # Initialize parameters: weights, means, and covariances
        self.weights = nn.Parameter(torch.ones(n_components) / n_components, requires_grad=True)  # Mixing coefficients
        self.means = nn.Parameter(torch.randn(n_components, n_features), requires_grad=True)  # Means of Gaussians
        self.covariances = nn.Parameter(torch.eye(n_features).repeat(n_components, 1, 1), requires_grad=True)  # Covariances
    
    def gaussian_pdf(self, X, mean, covariance):
        """Compute the probability density function of a Gaussian."""
        n_features = X.size(1)
        diff = X - mean
        
        # Compute the determinant and inverse of the covariance matrix
        det_cov = torch.det(covariance)
        inv_cov = torch.inverse(covariance)
        
        # Calculate exponent term
        exponent = -0.5 * (diff.unsqueeze(-2) @ inv_cov @ diff.unsqueeze(-1)).squeeze()
        
        # Calculate Gaussian PDF
        return (1.0 / ((2 * torch.pi) ** (n_features / 2) * det_cov.sqrt())) * torch.exp(exponent)
    
    def e_step(self, X):
        """Expectation step: compute the responsibilities."""
        likelihoods = torch.stack([self.gaussian_pdf(X, self.means[k], self.covariances[k]) for k in range(self.n_components)])
        weighted_likelihoods = self.weights.unsqueeze(1) * likelihoods
        responsibilities = weighted_likelihoods / (weighted_likelihoods.sum(dim=0) + 1e-9)
        return responsibilities
    
    def m_step(self, X, responsibilities):
        """Maximization step: update weights, means, and covariances."""
        Nk = responsibilities.sum(dim=1)
        self.weights.data = Nk / X.size(0)
        
        # Update means
        for k in range(self.n_components):
            self.means.data[k] = (responsibilities[k].unsqueeze(-1) * X).sum(dim=0) / Nk[k]
            
            # Update covariances
            diff = X - self.means[k]
            self.covariances.data[k] = (responsibilities[k].unsqueeze(-1) * diff.unsqueeze(-1) @ diff.unsqueeze(-2)).sum(dim=0) / Nk[k]
    
    def fit(self, X, n_iterations=100):
        """Fit the GMM to the data using the EM algorithm."""
        for i in range(n_iterations):
            responsibilities = self.e_step(X)
            self.m_step(X, responsibilities)
    
    def predict(self, X):
        """Predict cluster labels for the data."""
        responsibilities = self.e_step(X)
        return responsibilities.argmax(dim=0)

# Example usage:
n_samples, n_features, n_components = 100, 2, 3
X = torch.randn(n_samples, n_features)  # Generate some random data

gmm = GaussianMixtureModel(n_components=n_components, n_features=n_features)
gmm.fit(X, n_iterations=100)
labels = gmm.predict(X)

print("Cluster labels:", labels)
